## ハイパラ探索

In [1]:
%%capture
!pip install optuna

In [2]:
import optuna
import lightgbm as lgbm

In [ ]:
def objective(trial):
    param = {
        'boosting_type': 'gbdt',
        'objective': 'regression', #回帰
        #'objective': 'multiclass',
        #'num_class': 3,
        # カテゴリの試行を行うパラメータ
        #'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        # 整数値の試行を行うパラメータ
        'num_leavrs': trial.suggest_int('num_leaves', 10, 100),
        # 対数値の試行を行うパラメータ
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
        # 連続値の試行を行うパラメータ
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0, 1),
        # 離散値の試行を行うパラメータ(qは試行を行う間隔を調整する)
        #'min_child_samples': trial.suggest_discrete_uniform('min_child_samples', 1, 10, 1)
    }
    
    train_xy = lgbm.Dataset(train_x, train_y)
    val_xy = lgbm.Dataset(test_x, test_y, reference=train_xy)

    gbm = lgbm.train(param, train_xy,valid_sets = val_xy)
    
    pred_proba = gbm.predict(test_x)
    pred = np.argmax(pred_proba, axis=1)
    
    auc = roc_auc_score(test_y, pred_proba, multi_class = 'ovr')
    return auc


study = optuna.create_study(direction='minimize')
optuna.logging.disable_default_handler()
study.optimize(objective, n_trials=100)